In [1]:
import sys
sys.path.append(r"D:\往期\QHJ\ZERO")
import os
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pymysql
import pandas as pd
from tools import MallGoods,MALL_KEY,MYSQL_MALL_DIC
mg = MallGoods(MALL_KEY)



conn = pymysql.Connect(**MYSQL_MALL_DIC)
cursor = conn.cursor(pymysql.cursors.DictCursor)
cursor.execute('select * from goods_class')
goods_class = pd.DataFrame(cursor.fetchall())
goods_class_dic = dict(zip(goods_class['class_name'],goods_class['class_id']))



goods_path = r"D:\奇货居\素材\商城图片素材\商品信息.xlsx"


file_o = pd.ExcelFile(goods_path)

In [2]:
goods_df = file_o.parse("商品详情",)

not_uploaded_df = file_o.parse('SPU')[:2]
not_uploaded_df.loc[0,'goods_id'] = 27
not_uploaded_df.loc[1,'goods_id'] = 33


In [3]:
goods_class_dic

{'背包': 1,
 '数码产品': 2,
 '美容护肤': 3,
 '床上用品': 4,
 '电瓶车': 5,
 '智能手机': 6,
 '饰品': 7,
 '电器': 8,
 '儿童': 9}

In [4]:
# 上传新商品 图片
for index,row in not_uploaded_df.iterrows():
    goods_id = row['goods_id']
    img_dir = os.path.join(row['img_dir'],"轮播")
    mg.up_imgs(goods_id,img_dir,"goods_slide")

    img_dir = os.path.join(row['img_dir'],"详情")
    
    mg.up_imgs(goods_id,img_dir,"goods_info")
    
    # 上架
    mg.set_shelf(goods_id,1)
    
    
    # 修改 goods 描述信息
    data = {
            "goods_name": row['商品名简称'],
            "class_id": goods_class_dic[row['系统分类']],
            "goods_info": row['商品名简称'],
            "goods_id": goods_id,
    }
    mg.set_goods(data)
    
    
    # 设置 goods 属性
    data = {
        "goods_id":goods_id,
        "sort":0,
        'send_price':15,
        'goods_price':row['市场价']
    }

    mg.set_good_opt(data)

{'code': 200, 'err_msg': '操作成功', 'data': '', 'goods_id': 27}
{'code': 200, 'err_msg': '操作成功', 'data': '', 'goods_id': 33}


In [5]:
mg.set_shelf(27,1)

goods_id:27 设为上架


{'code': 200, 'err_msg': '操作成功', 'data': ''}

In [23]:

# 规格 细节属性
not_uploaded_free_df = not_uploaded_df[not_uploaded_df['goods_type'] != 'free']
rename_dic = {
    "商品ID":"attr_goods_code",
    "规格":"attr_name",
    '售价':"attr_price",
    '单位' :"attr_unit",
    "市场价" :'attr_old_price'
}
goods_df_r = goods_df[['商品ID','规格','售价','单位','市场价','attr_stock','SPUID']]

not_uploaded_free_df = pd.merge(not_uploaded_free_df[['SPUID','goods_id']],goods_df_r,on='SPUID')


not_uploaded_free_df.rename(columns=rename_dic,inplace=True)
not_uploaded_free_df['attr_stock'][not_uploaded_free_df['attr_stock'].isnull()] = 500

mg.set_attrs(not_uploaded_free_df)


# 特殊处理

# 批量你要修改的商品
# def update_goods_attr(goods_id_set):
#     up_data = not_uploaded_free_df[not_uploaded_free_df['goods_id'].isin(goods_id_set)]

#     mg.set_attrs(up_data)

# update_goods_attr({1})

访问 27
{'code': 200, 'err_msg': '操作成功！', 'data': ''} 27
访问 33
{'code': 200, 'err_msg': '操作成功！', 'data': ''} 33


In [18]:
not_uploaded_df

,序号,货品编号,商品ID,goods_id,attr_stock,系统分类,类别,规格模式,规格,售价,...,交易编码,发货商,发货商ID,供应商,规格编码,供应商ID,商品编码,SPUID,goods_type,img_dir
0,1,NaN,S0001P0001C0001P01E01,27,NaN,背包,NaN,颜色,蓝,259,...,E01,傻傻,S0001,Delsey,P01,P0001,C0001,S0001P0001C0001,NaN,D:\奇货居\素材\商城图片素材\傻傻\Delsey\00001560002双肩休闲包
1,2,NaN,S0001P0001C0002P01E01,33,NaN,背包,NaN,颜色,黑,215,...,E01,傻傻,S0001,Delsey,P01,P0001,C0002,S0001P0001C0002,NaN,D:\奇货居\素材\商城图片素材\傻傻\Delsey\70371560011商务电脑背包


In [16]:
not_uploaded_df['商品名简称']

0     Delsey 双肩休闲包
1    Delsey 商务电脑背包
Name: 商品名简称, dtype: object

In [4]:
goods_df

,序号,货品编号,商品ID,goods_id,attr_stock,系统分类,类别,规格模式,规格,售价,...,交易编码,发货商,发货商ID,供应商,规格编码,供应商ID,商品编码,SPUID,goods_type,img_dir
0,1,NaN,S0001P0001C0001P01E01,1,NaN,背包,NaN,颜色,蓝,259,...,E01,傻傻,S0001,Delsey,P01,P0001,C0001,S0001P0001C0001,NaN,D:\奇货居\素材\商城图片素材\傻傻\Delsey\00001560002双肩休闲包
1,2,NaN,S0001P0001C0001P02E01,1,NaN,背包,NaN,颜色,红,259,...,E01,傻傻,S0001,Delsey,P02,P0001,C0001,S0001P0001C0001,NaN,D:\奇货居\素材\商城图片素材\傻傻\Delsey\00001560002双肩休闲包
2,3,NaN,S0001P0001C0001P03E01,1,NaN,背包,NaN,颜色,灰,259,...,E01,傻傻,S0001,Delsey,P03,P0001,C0001,S0001P0001C0001,NaN,D:\奇货居\素材\商城图片素材\傻傻\Delsey\00001560002双肩休闲包
3,4,NaN,S0001P0001C0002P01E01,2,NaN,背包,NaN,颜色,黑,215,...,E01,傻傻,S0001,Delsey,P01,P0001,C0002,S0001P0001C0002,NaN,D:\奇货居\素材\商城图片素材\傻傻\Delsey\70371560011商务电脑背包
4,5,NaN,S0001P0001C0002P02E01,2,NaN,背包,NaN,颜色,红,215,...,E01,傻傻,S0001,Delsey,P02,P0001,C0002,S0001P0001C0002,NaN,D:\奇货居\素材\商城图片素材\傻傻\Delsey\70371560011商务电脑背包
5,6,NaN,S0001P0001C0003P01E01,3,NaN,背包,NaN,颜色,黑,295,...,E01,傻傻,S0001,Delsey,P01,P0001,C0003,S0001P0001C0003,NaN,D:\奇货居\素材\商城图片素材\傻傻\Delsey\70371760022商务旅行背包
6,7,NaN,S0001P0001C0003P02E01,3,NaN,背包,NaN,颜色,灰,295,...,E01,傻傻,S0001,Delsey,P02,P0001,C0003,S0001P0001C0003,NaN,D:\奇货居\素材\商城图片素材\傻傻\Delsey\70371760022商务旅行背包
7,8,NaN,S0001P0001C0003P03E01,3,NaN,背包,NaN,颜色,蓝,295,...,E01,傻傻,S0001,Delsey,P03,P0001,C0003,S0001P0001C0003,NaN,D:\奇货居\素材\商城图片素材\傻傻\Delsey\70371760022商务旅行背包
8,9,NaN,S0001P0002C0001P01E01,6,NaN,床上用品,NaN,NaN,经典套件,359,...,E01,傻傻,S0001,LOVO,P01,P0002,C0001,S0001P0002C0001,NaN,D:\奇货居\素材\商城图片素材\傻傻\LOVO\LOVO 经典套装
9,10,NaN,S0001P0002C0002P01E01,10,NaN,床上用品,NaN,NaN,魅力格调,275,...,E01,傻傻,S0001,LOVO,P01,P0002,C0002,S0001P0002C0002,NaN,D:\奇货居\素材\商城图片素材\傻傻\LOVO\LOVO 魅力格调
